<a href="https://colab.research.google.com/github/kcarstons/backend_appraisals/blob/main/Florence2_inferencing_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install timm flash_attn einops
!pip install -q -U transformers==4.37.2
!pip install pypdfium2

In [8]:
import requests
import torch
from transformers import BitsAndBytesConfig, AutoProcessor, AutoModelForCausalLM
import pypdfium2 as pdfium

def extract_images(file_name):
    """
    Extract all images from a pdf and store in a list.
    """
    all_images = []
    pdf = pdfium.PdfDocument(file_name)

    for i in range(len(pdf)):
        page = pdf[i]

        for obj in page.get_objects():
            if obj.type == 3:
                image = obj.get_bitmap().to_pil()
                all_images.append(image)

    return all_images



In [9]:
extracted_images_sf = extract_images("/content/appraisal_sf.pdf")
subject_images_sf = extracted_images_sf[3:12]
labels_sf = [ 'subject front', 'subject rear', 'subject street', 'kitchen', 'nook', 'living/dining', 'bedroom', 'bathroom', 'bathroom']
labeled_images_sf = [{'image':img, 'label': lbl} for img, lbl in zip(subject_images_sf, labels_sf)]


In [10]:
extracted_images_fha = extract_images("/content/fha_appraisal.pdf")
subject_images_fha = extracted_images_fha[11:18]
labels_fha = ['Subject Front', 'Subject Rear', 'Subject Street', 'Interior', 'Interior', 'Interior', 'Interior']
labeled_images_fha = [{'image':img, 'label': lbl} for img, lbl in zip(subject_images_fha, labels_fha)]



In [ ]:


model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

prompt = "<MORE_DETAILED_CAPTION>"

fha_captions = []
sf_captions = []



def florence_answer_prompt(pil_image, prompt):

    inputs = processor(images=pil_image, text = prompt, return_tensors="pt")

    generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      do_sample=False,
      num_beams=3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    parsed_answer = processor.post_process_generation(generated_text, task="<MORE_DETAILED_CAPTION>", image_size=(pil_image.width, pil_image.height))

    return parsed_answer






In [12]:
# for item in labeled_images_sf:
#     pil_image = item['image']
#     label = item['label']
#     parsed_answer = florence_answer_prompt(pil_image, prompt)
#     sf_captions.append(parsed_answer)

# sf_captions

for item in labeled_images_sf:
    pil_image = item['image']
    label = item['label']
    response = florence_answer_prompt(pil_image, prompt)
    print(response)



{'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern house with a wooden exterior and a sloping roof. The house has a white exterior with large windows and a balcony on the second floor. The balcony has a railing and there is a wooden walkway leading up to it. The walkway is surrounded by a wooden fence and there are trees and bushes in the background. The sky is blue with some clouds and the overall atmosphere of the image is peaceful and serene.'}
{'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern house. The house is made of corrugated metal and has a sloping roof. It has multiple windows and doors, and a staircase leading up to the entrance. The exterior of the house is painted in a dark brown color, and there is a small garden in front of it with shrubs and bushes. The sky is blue and there are a few clouds in the background. The overall atmosphere of the image is peaceful and serene.'}
{'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a 

In [ ]:
#BASE MODEL (4mins no quant fewer resources)
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern house with a wooden exterior and a sloping roof. The house has a white exterior with large windows and a balcony on the second floor. The balcony has a railing and there is a wooden walkway leading up to it. The walkway is surrounded by a wooden fence and there are trees and bushes in the background. The sky is blue with some clouds and the overall atmosphere of the image is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern house. The house is made of corrugated metal and has a sloping roof. It has multiple windows and doors, and a staircase leading up to the entrance. The exterior of the house is painted in a dark brown color, and there is a small garden in front of it with shrubs and bushes. The sky is blue and there are a few clouds in the background. The overall atmosphere of the image is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a row of houses in a residential area. The houses are made of wood and have a modern design with a sloping roof and large windows. The front of the houses has a small balcony with a railing and a small garden with shrubs and trees. The sky is blue and there are a few clouds in the background. The street is lined with trees and there is a sidewalk on the right side of the image. The overall atmosphere of the scene is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern kitchen with white cabinets and a wooden table in the center. The kitchen has a large window on the right side, allowing natural light to enter the space. The walls are painted in a light green color, and the floor is made of white tiles. On the left side of the image, there is a white countertop with a sink and a gas cooktop. Above the countertop, there are two pendant lights hanging from the ceiling. The overall style of the kitchen is minimalistic and contemporary.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern and minimalistic living room and kitchen area. The room has a white ceiling with recessed lighting and a wooden floor. On the left side of the image, there is a white refrigerator and a green wall with a picture hanging on it. Next to the refrigerator, there are two pendant lights hanging from the ceiling. In the center of the room, there has a wooden table with a few items on it, including a vase with a green plant and a white vase. The walls are painted in a light blue color, and there are a few black and white furniture pieces scattered around the room. A window with white curtains is visible in the background, letting in natural light.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern living room with a large window on the left side. The room has a gray sofa with blue throw pillows, a round coffee table with a black metal frame, and two green side tables. On the right side of the room, there is a wooden dining table with four chairs. The floor is made of light-colored tiles, and the walls are painted white. The window has large panes of glass, allowing natural light to enter the room. Through the window, we can see a view of trees and a building outside. The overall atmosphere of the space is bright and airy.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a corner of a room with a wooden floor and white walls. The room has a large window on the left side, allowing natural light to enter the space. On the right side, there is a sliding glass door that leads to a balcony with a view of trees and a fence. In the center of the room, there are two wooden benches with gray cushions and a wooden headboard. Above the benches is a wooden pendant light hanging from the ceiling. The floor is made of light-colored tiles.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern bathroom with a minimalist design. The walls are painted white and the floor is made of dark grey tiles. On the left side of the image, there is a white shelving unit with three shelves. The shelves are filled with folded towels and a potted plant. Next to the shelves, there are two white toilet seats. The shower area has a glass door with a silver handle and a showerhead. The overall color scheme of the bathroom is white, gray, and black.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a small bathroom with a white vanity and a white sink. The vanity has two drawers and a mirror above it. On the right side of the vanity, there is a white shelf with a potted plant on top. The walls are painted in a light blue color and the floor is covered with a gray carpet. There is a door on the left side and a window on the far wall. The overall style of the bathroom is modern and minimalistic.'}



#LARGE MODEL (7mins no quant)

# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern house with a wooden exterior. The house has a sloping roof and large windows that let in natural light. The front of the house is covered with a white wall and has a wooden deck with a railing. The deck is surrounded by a wooden fence and there are trees and bushes in the foreground. In the background, there are other houses and buildings visible. The sky is blue with some clouds. The overall atmosphere of the image is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern building with a unique design. The building is made of wood and has a sloping roof. It has multiple windows and doors, and a staircase leading up to the entrance. The entrance is covered with a metal railing, and there is a small garden in front of the building with shrubs and trees. The sky is blue and there are a few clouds in the background. The overall atmosphere of the image is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a residential area. It shows a row of houses on a street, with a wooden building on the left side of the image. The building has a sloping roof and a large window on the front. The street is lined with trees and shrubs, and there is a small pond in the foreground. The sky is blue and there are a few clouds in the distance. The houses are painted in different colors and styles, with some having white walls and others having brown roofs. The overall atmosphere of the scene is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern kitchen and dining area. The kitchen has white cabinets and a wooden countertop with a sink and a gas stove. There is a large island in the center of the room with a wooden table and chairs. Above the table, there are two pendant lights hanging from the ceiling. The walls are painted in a light green color and there is a window on the right side of the image that lets in natural light. The floor is made of light-colored tiles. The overall style of the kitchen is minimalistic and contemporary.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern kitchen and living room in a small apartment. The kitchen has a wooden countertop with a sink and a potted plant on it. The walls are painted in a light green color and there is a large window on the right side of the image that lets in natural light. On the left side, there are white cabinets and a white refrigerator. Above the kitchen, there is an orange pendant light hanging from the ceiling. The floor is made of light-colored tiles. The room has a gray sofa and a gray armchair in the background. The overall style of the room is minimalistic and contemporary.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern living room with a large window on the left side. The room has a gray sofa with blue throw pillows and a round coffee table in the center. On the right side, there is a wooden dining table with four chairs around it. The floor is made of light-colored tiles and the walls are painted white. The ceiling is high and has a chandelier hanging from it. Through the large window, one can see a balcony with a view of trees and a building. The overall atmosphere of the room is bright and airy.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern living room with a large window on the left side. The window has a sliding glass door that leads to a balcony with a view of trees and a fence. On the right side of the room, there is a wooden bench with a gray cushion and a white coffee table in front of it. Above the bench, there are two pendant lights hanging from the ceiling. The walls are painted in a light beige color and the floor is made of light-colored tiles. The room appears to be empty, with no furniture or decorations in sight.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern bathroom with a minimalist design. The walls are painted white and the floor is covered with black tiles. On the left side of the image, there is a white ladder shelf with folded towels and a potted plant on top. Next to the ladder shelf, there are two white toilet seats. The shower area is enclosed by a glass door with a silver handle and a showerhead. The overall color scheme of the bathroom is white and gray.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern bathroom with a white color scheme. The walls are painted white and there is a large mirror above the sink. The sink has a white countertop with a silver faucet and two drawers below it. On the right side of the sink, there are two white shelves with a vase of purple flowers on top. The floor is made of gray tiles. The bathroom has a door on the left side and a small window on the right. The overall style of the bathroom is minimalistic and contemporary.'}



In [13]:
for item in labeled_images_fha:
    pil_image = item['image']
    label = item['label']
    response = florence_answer_prompt(pil_image, prompt)
    print(response)


{'<MORE_DETAILED_CAPTION>': 'The image shows a single-story house with a gray exterior and a red door. The house is located on a street with a driveway in front of it. There are trees and bushes surrounding the house, and a few cars parked on the right side of the street. The sky is blue and the weather appears to be sunny and pleasant.'}
{'<MORE_DETAILED_CAPTION>': 'The image shows a two-story house with a white exterior and a sloping roof. The house is located on a grassy lawn with trees in the background. There are several windows on the front of the house, and a chimney on the roof. A few people can be seen walking around the house. The sky is blue and the overall atmosphere is peaceful and serene.'}
{'<MORE_DETAILED_CAPTION>': 'The image shows a street with a few cars parked on the right side of the road. The street is lined with trees and bushes on both sides, and there are houses on the left side. The sky is blue with some clouds, and the street appears to be empty. There is a p

In [ ]:
#BASE MODEL (4mins no quant fewer resources)
# {'<MORE_DETAILED_CAPTION>': 'The image shows a single-story house with a gray exterior and a red door. The house is located on a street with a driveway in front of it. There are trees and bushes surrounding the house, and a few cars parked on the right side of the street. The sky is blue and the weather appears to be sunny and pleasant.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a two-story house with a white exterior and a sloping roof. The house is located on a grassy lawn with trees in the background. There are several windows on the front of the house, and a chimney on the roof. A few people can be seen walking around the house. The sky is blue and the overall atmosphere is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a street with a few cars parked on the right side of the road. The street is lined with trees and bushes on both sides, and there are houses on the left side. The sky is blue with some clouds, and the street appears to be empty. There is a power line running along the street, and a few power lines can be seen in the distance. The image is taken from a low angle, looking down the street towards the houses.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a kitchen with white cabinets and black countertops. The kitchen has a black dishwasher and a white refrigerator on the left side. On the right side, there is a window with a green and white checkered curtain. The floor is made of light-colored tiles. There are various kitchen utensils and appliances scattered around the kitchen, including a sink, a stove, and a dishwasher. The walls are painted white and there are a few framed pictures hanging on the wall.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows an empty room with white walls and wooden flooring. On the left side of the room, there is a window with green blinds. Next to the window, there are two black speakers on a black pedestal. A white lamp is placed on one of the speakers. The room appears to be empty, with no furniture or decorations in sight.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a bedroom with a single bed in the center. The bed is covered with a green and white checkered comforter and there are several pillows and blankets scattered on top. On the left side of the bed, there is a window with white blinds and a wooden headboard. The walls are painted in a light blue color and the floor is made of wood. The room appears to be well-lit with natural light coming in from the window.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a small bathroom with a white toilet and a blue bathtub. The walls are painted in a light blue color and there is a window with white blinds on the right side of the image. On the left side, there are white cabinets with a sink and a mirror above it. The floor is covered with a gray carpet. The door is white and has a gold handle. There is a white towel hanging on the wall next to the toilet.'}



#LARGE MODEL (7mins no quant)

# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern house with a wooden exterior. The house has a sloping roof and large windows that let in natural light. The front of the house is covered with a white wall and has a wooden deck with a railing. The deck is surrounded by a wooden fence and there are trees and bushes in the foreground. In the background, there are other houses and buildings visible. The sky is blue with some clouds. The overall atmosphere of the image is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern building with a unique design. The building is made of wood and has a sloping roof. It has multiple windows and doors, and a staircase leading up to the entrance. The entrance is covered with a metal railing, and there is a small garden in front of the building with shrubs and trees. The sky is blue and there are a few clouds in the background. The overall atmosphere of the image is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a residential area. It shows a row of houses on a street, with a wooden building on the left side of the image. The building has a sloping roof and a large window on the front. The street is lined with trees and shrubs, and there is a small pond in the foreground. The sky is blue and there are a few clouds in the distance. The houses are painted in different colors and styles, with some having white walls and others having brown roofs. The overall atmosphere of the scene is peaceful and serene.'}
# {'<MORE_DETAILED_CAPTION>': 'The image is a 3D rendering of a modern kitchen and dining area. The kitchen has white cabinets and a wooden countertop with a sink and a gas stove. There is a large island in the center of the room with a wooden table and chairs. Above the table, there are two pendant lights hanging from the ceiling. The walls are painted in a light green color and there is a window on the right side of the image that lets in natural light. The floor is made of light-colored tiles. The overall style of the kitchen is minimalistic and contemporary.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern kitchen and living room in a small apartment. The kitchen has a wooden countertop with a sink and a potted plant on it. The walls are painted in a light green color and there is a large window on the right side of the image that lets in natural light. On the left side, there are white cabinets and a white refrigerator. Above the kitchen, there is an orange pendant light hanging from the ceiling. The floor is made of light-colored tiles. The room has a gray sofa and a gray armchair in the background. The overall style of the room is minimalistic and contemporary.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern living room with a large window on the left side. The room has a gray sofa with blue throw pillows and a round coffee table in the center. On the right side, there is a wooden dining table with four chairs around it. The floor is made of light-colored tiles and the walls are painted white. The ceiling is high and has a chandelier hanging from it. Through the large window, one can see a balcony with a view of trees and a building. The overall atmosphere of the room is bright and airy.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern living room with a large window on the left side. The window has a sliding glass door that leads to a balcony with a view of trees and a fence. On the right side of the room, there is a wooden bench with a gray cushion and a white coffee table in front of it. Above the bench, there are two pendant lights hanging from the ceiling. The walls are painted in a light beige color and the floor is made of light-colored tiles. The room appears to be empty, with no furniture or decorations in sight.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern bathroom with a minimalist design. The walls are painted white and the floor is covered with black tiles. On the left side of the image, there is a white ladder shelf with folded towels and a potted plant on top. Next to the ladder shelf, there are two white toilet seats. The shower area is enclosed by a glass door with a silver handle and a showerhead. The overall color scheme of the bathroom is white and gray.'}
# {'<MORE_DETAILED_CAPTION>': 'The image shows a modern bathroom with a white color scheme. The walls are painted white and there is a large mirror above the sink. The sink has a white countertop with a silver faucet and two drawers below it. On the right side of the sink, there are two white shelves with a vase of purple flowers on top. The floor is made of gray tiles. The bathroom has a door on the left side and a small window on the right. The overall style of the bathroom is minimalistic and contemporary.'}
